In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Patient-Recovery-Prediction-Challenge/sample_submission.csv
/kaggle/input/Patient-Recovery-Prediction-Challenge/train.csv
/kaggle/input/Patient-Recovery-Prediction-Challenge/test.csv


In [75]:
import pandas as pd

# Load the training dataset
train_df = pd.read_csv("/kaggle/input/Patient-Recovery-Prediction-Challenge/train.csv")

# View the first few rows
train_df.head()


,Id,Therapy Hours,Initial Health Score,Lifestyle Activities,Average Sleep Hours,Follow-Up Sessions,Recovery Index
0,9255,5,49,No,7,5,36
1,1562,2,48,Yes,7,6,25
2,1671,2,81,No,7,2,59
3,6088,2,46,No,6,1,22
4,6670,8,47,No,9,0,40


In [76]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [77]:
# Remove the ID column
train_df = train_df.drop(columns=['Id'])
# train_df = train_df.drop(columns=['Lifestyle Activities'])
train_df["Lifestyle Activities_Yes"] = train_df["Lifestyle Activities"].apply(lambda x: 1 if x == "Yes" else 0)
train_df = train_df.drop(columns=["Lifestyle Activities"])
# Separate features (X) and target (y)
X = train_df.drop(columns=['Recovery Index'])
y = train_df['Recovery Index']

train_df.head()

,Therapy Hours,Initial Health Score,Average Sleep Hours,Follow-Up Sessions,Recovery Index,Lifestyle Activities_Yes
0,5,49,7,5,36,0
1,2,48,7,6,25,1
2,2,81,7,2,59,0
3,2,46,6,1,22,0
4,8,47,9,0,40,0


In [78]:
# X = pd.get_dummies(X, drop_first=True)
# X.head()

In [79]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42
)


In [80]:
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [81]:
y_pred = model.predict(X_val)

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Mean Squared Error: 4.0595
R² Score: 0.9892


In [82]:
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', ascending=False)

coefficients.head(10)


,Feature,Coefficient
0,Therapy Hours,2.857617
1,Initial Health Score,1.016582
4,Lifestyle Activities_Yes,0.633299
2,Average Sleep Hours,0.488614
3,Follow-Up Sessions,0.192843


In [83]:
test_df = pd.read_csv("/kaggle/input/Patient-Recovery-Prediction-Challenge/test.csv")


In [84]:
# Save ID separately for the output
test_ids = test_df['Id'] if 'Id' in test_df.columns else test_df['ID']

# Drop ID column before prediction
test_features = test_df.drop(columns=['Id', 'ID'], errors='ignore')


In [85]:
test_features = pd.get_dummies(test_features, drop_first=True)

# Align columns with training data
test_features = test_features.reindex(columns=X.columns, fill_value=0)


In [86]:
test_predictions = model.predict(test_features)


In [87]:
submission = pd.DataFrame({
    'Id': test_ids,
    'Recovery Index': test_predictions
})

submission.to_csv("/kaggle/working/submission.csv", index=False)
print("✅ Submission file created successfully!")
submission.head()


✅ Submission file created successfully!


,Id,Recovery Index
0,6253,54.719621
1,4685,22.601292
2,1732,47.942506
3,4743,31.341359
4,4522,42.986464
